# LSTM Stock Predictor Using Fear and Greed Index

In this notebook, you will build and train a custom LSTM RNN that uses a 10 day window of Bitcoin fear and greed index values to predict the 11th day closing price. 

You will need to:

1. Prepare the data for training and testing
2. Build and train a custom LSTM RNN
3. Evaluate the performance of the model

## Data Preparation

In this section, you will need to prepare the training and testing data for the model. The model will use a rolling 10 day window to predict the 11th day closing price.

You will need to:
1. Use the `window_data` function to generate the X and y values for the model.
2. Split the data into 70% training and 30% testing
3. Apply the MinMaxScaler to the X and y values
4. Reshape the X_train and X_test data for the model. Note: The required input format for the LSTM is:

```python
reshape((X_train.shape[0], X_train.shape[1], 1))
```

In [69]:
import numpy as np
import pandas as pd
import hvplot.pandas

In [70]:
# Set the random seed for reproducibility
# Note: This is for the homework solution, but it is good practice to comment this out and run multiple experiments to evaluate your model
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(2)

In [71]:
# Load the fear and greed sentiment data for Bitcoin
df = pd.read_csv('btc_sentiment.csv', index_col="date", infer_datetime_format=True, parse_dates=True)
df = df.drop(columns="fng_classification")
df.head()

,fng_value
date,
2019-07-29,19
2019-07-28,16
2019-07-27,47
2019-07-26,24
2019-07-25,42


In [72]:
# Load the historical closing prices for Bitcoin
df2 = pd.read_csv('btc_historic.csv', index_col="Date", infer_datetime_format=True, parse_dates=True)['Close']
df2 = df2.sort_index()
df2.tail()

Date
2019-07-25    9882.429688
2019-07-26    9847.450195
2019-07-27    9478.320313
2019-07-28    9531.769531
2019-07-29    9529.889648
Name: Close, dtype: float64

In [73]:
# Join the data into a single DataFrame
df = df.join(df2, how="inner")
df.tail()

,fng_value,Close
2019-07-25,42,9882.429688
2019-07-26,24,9847.450195
2019-07-27,47,9478.320313
2019-07-28,16,9531.769531
2019-07-29,19,9529.889648


In [74]:
df.head()

,fng_value,Close
2018-02-01,30,9114.719727
2018-02-02,15,8870.820313
2018-02-03,40,9251.269531
2018-02-04,24,8218.049805
2018-02-05,11,6937.080078


In [75]:
# This function accepts the column number for the features (X) and the target (y)
# It chunks the data up with a rolling window of Xt-n to predict Xt
# It returns a numpy array of X any y
def window_data(df, window, feature_col_number, target_col_number):
    X = []
    y = []
    for i in range(len(df) - window - 1):
        features = df.iloc[i:(i + window), feature_col_number]
        target = df.iloc[(i + window), target_col_number]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1, 1)

In [76]:
# Predict Closing Prices using a 10 day window of previous fng values
# Then, experiment with window sizes anywhere from 1 to 10 and see how the model performance changes
window_size = 1

# Column index 0 is the 'fng_value' column
# Column index 1 is the `Close` column
feature_column = 0
target_column = 1
X, y = window_data(df, window_size, feature_column, target_column)

In [77]:
# Use 70% of the data for training and the remaineder for testing
split = int(0.7 * len(X))

# X train and test data
X_train = X[: split - 1]
X_test = X[split:]

# y train and test data
y_train = y[: split - 1]
y_test = y[split:]

In [78]:
from sklearn.preprocessing import MinMaxScaler
# Use the MinMaxScaler to scale data between 0 and 1.
scaler = MinMaxScaler()

# fit the scaler to X
scaler.fit(X)

# scale the X train and test data 
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# fit the scaler to y
scaler.fit(y)

# scale the y train and test data 
y_train = scaler.transform(y_train)
y_test = scaler.transform(y_test)


In [79]:
# Reshape the features for the model
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

---

## Build and Train the LSTM RNN

In this section, you will design a custom LSTM RNN and fit (train) it using the training data.

You will need to:
1. Define the model architecture
2. Compile the model
3. Fit the model to the training data

### Hints:
You will want to use the same model architecture and random seed for both notebooks. This is necessary to accurately compare the performance of the FNG model vs the closing price model. 

In [80]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [81]:
# Build the LSTM model. 
# The return sequences need to be set to True if you are adding additional LSTM layers, but 
# You don't have to do this for the final layer. 
# Note: The dropouts help prevent overfitting
# Note: The input shape is the number of time steps and the number of indicators
# Note: Batching inputs has a different input shape of Samples/TimeSteps/Features

lstm_model = Sequential()

number_units = 30
dropout_fraction = 0.2

# LSTM model Layer 1
lstm_model.add(LSTM(units = number_units, return_sequences = True, input_shape = (X_train.shape[1],1)))
lstm_model.add(Dropout(dropout_fraction))

# LSTM model Layer 2
lstm_model.add(LSTM(units = number_units, return_sequences = True))
lstm_model.add(Dropout(dropout_fraction))

# LSTM model Layer 3
lstm_model.add(LSTM(units = number_units, return_sequences = False))
lstm_model.add(Dropout(dropout_fraction))

# LSTM model Output
lstm_model.add(Dense(1))

In [82]:
# Compile the model
lstm_model.compile(optimizer="adam", loss="mean_squared_error")

In [83]:
# Summarize the model
lstm_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_9 (LSTM)                (None, 1, 30)             3840      
_________________________________________________________________
dropout_9 (Dropout)          (None, 1, 30)             0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 1, 30)             7320      
_________________________________________________________________
dropout_10 (Dropout)         (None, 1, 30)             0         
_________________________________________________________________
lstm_11 (LSTM)               (None, 30)                7320      
_________________________________________________________________
dropout_11 (Dropout)         (None, 30)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                

In [84]:
# Train the model
# Use at least 10 epochs
# Do not shuffle the data
# Experiement with the batch size, but a smaller batch size is recommended
lstm_model.fit(X_train, y_train, epochs=10, shuffle=False, batch_size=2, verbose=1)

Epoch 1/10
189/189 [==============================] - 6s 3ms/step - loss: 0.0804
Epoch 2/10
189/189 [==============================] - 1s 3ms/step - loss: 0.0444
Epoch 3/10
189/189 [==============================] - 1s 4ms/step - loss: 0.0422
Epoch 4/10
189/189 [==============================] - 1s 4ms/step - loss: 0.0424
Epoch 5/10
189/189 [==============================] - 1s 5ms/step - loss: 0.0407
Epoch 6/10
189/189 [==============================] - 1s 4ms/step - loss: 0.0402
Epoch 7/10
189/189 [==============================] - 1s 4ms/step - loss: 0.0385
Epoch 8/10
189/189 [==============================] - 1s 3ms/step - loss: 0.0373
Epoch 9/10
189/189 [==============================] - 1s 3ms/step - loss: 0.0374
Epoch 10/10
189/189 [==============================] - 1s 4ms/step - loss: 0.0378


---

## Model Performance

In this section, you will evaluate the model using the test data. 

You will need to:
1. Evaluate the model using the `X_test` and `y_test` data.
2. Use the X_test data to make predictions
3. Create a DataFrame of Real (y_test) vs predicted values. 
4. Plot the Real vs predicted values as a line chart

### Hints
Remember to apply the `inverse_transform` function to the predicted and y_test values to recover the actual closing prices.

In [85]:
# Evaluate the model
lstm_model.evaluate(X_test, y_test)

6/6 [==============================] - 1s 2ms/step - loss: 0.0961


0.09605463594198227

In [86]:
# Make some predictions
predicted = lstm_model.predict(X_test)

In [87]:
# Recover the original prices instead of the scaled version
predicted_prices = scaler.inverse_transform(predicted)
real_prices = scaler.inverse_transform(y_test.reshape(-1, 1))

In [88]:
# Create a DataFrame of Real and Predicted values
stocks = pd.DataFrame({
    "Real": real_prices.ravel(),
    "Predicted": predicted_prices.ravel()
}, index = df.index[-len(real_prices): ]) 
stocks.head()

,Real,Predicted
2019-02-17,3670.919922,5169.827637
2019-02-18,3670.919922,5122.474121
2019-02-19,3912.570068,5051.854980
2019-02-20,3924.239990,5648.503418
2019-02-21,3974.050049,5696.194336


In [89]:
# Plot the real vs predicted values as a line chart
stocks.hvplot.line(xlabel="Date",
                  ylabel="Price",
                  title = "real vs predicted values")

:NdOverlay   [Variable]
   :Curve   [index]   (value)